# Курсовая работа
Тема 1 Решение систем линейных алгебраических уравнений с симметричными разреженными матрицами большой размерности. Метод сопряженных градиентов.


Упаковать

Сравнить с гаусс по времени

In [1]:
import scipy.sparse as sc
from scipy import stats
from scipy.linalg import solve
from numpy.random import default_rng, rand
import numpy as np
from scipy.sparse.linalg import cg as lcg
from sklearn.datasets import make_sparse_spd_matrix
from numpy.linalg import solve, norm
import matplotlib.pyplot as plt
import math
from scipy.sparse import csr_matrix
import time
import copy

#### Создание  разряженной симметрической матрицы и правой части СЛАУ

In [2]:
def matr(size):
    A = make_sparse_spd_matrix(size,alpha=0.95)
    return(A)

In [3]:
def xb(size):
    b = np.random.rand(size,1)
    x = (np.zeros(size)+1.0).reshape(-1, 1)
    return x, b

#### Метод сопряженных градиентов (CG)

In [4]:
def norma(r):
    res = 0
    for i in r:
        res += r*r
    return math.sqrt(sum(res))

In [5]:
def norma_s(r):
    res = r.multiply(r)
    res = res.sum()
    return math.sqrt(res)

In [6]:
def cg_s(A,b,x):
    start_time = time.time()
    r = b - A @ x
    p = r
    rsold = r.T @ r
    k = 0
    time2 = 0
    while True:
        Ap = A @ p
        alpha = rsold / (p.T @ Ap)
        time1 = time.time()
        alpha = csr_matrix(alpha)
        time2 += (time.time() -time1)
        x = x + p * alpha
        r = r - Ap * alpha
        rsnew = r.T @ r
        if norma_s(r) < 0.001:
            print("Количество шагов :", k)
            print(time.time()- start_time - time2)

            return x,k, time.time()- start_time - time2
        time3 = time.time()
        r_del = csr_matrix(rsnew / rsold)
        time2 += (time.time() - time3)
        p = r + p* r_del
        rsold = rsnew
        k = k + 1

In [7]:
def cg(A,b,x):
    start_time = time.time()
    r = b - A @ x
    p = r
    rsold = r.T @ r
    k = 0
    while True:
        Ap = A @ p
        alpha = rsold / (p.T @ Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rsnew = r.T @ r
        if norma(r) < 0.001:
            print("Количество шагов :", k)
            print(time.time()- start_time)
            return x,k, time.time()- start_time
        p = r + (rsnew / rsold) * p
        rsold = rsnew
        k = k + 1

##### Проверка алгоритма

In [8]:
A = np.array([[4,1],[1,3]])
b = np.array([1,2]).reshape(-1, 1)
x = np.array([2,1]).reshape(-1, 1)

In [9]:
(np.zeros(3)+1.0).reshape(-1, 1)

array([[1.],
       [1.],
       [1.]])

In [10]:
np.array([2,1]).reshape(-1, 1)

array([[2],
       [1]])

In [11]:
r = b - A @ x
r


array([[-8],
       [-3]])

In [12]:
cg(A,b,x)

Количество шагов : 1
0.004196882247924805


(array([[0.09090909],
        [0.63636364]]),
 1,
 0.004220724105834961)

In [13]:
lcg(A,b,x)

(array([0.09090909, 0.63636364]), 0)

#### Результат

In [14]:
def ans(size = 5):
    A = matr(size)
    x,b = xb(size)
    CG1_x, CG1_k, CG1_t =cg(A,b,x)
    CG2 = lcg(A,b,x)[0]
    A,b, x = csr_matrix(A), csr_matrix(b), csr_matrix(x)
    CG3_x, CG3_k, CG3_t = cg_s(A,b,x)
    j = 0
    summa = 0
    while j < size:
        summa += (CG2[j]-CG1_x[j])**2
        j = j + 1
    mer1 = summa/size
    cg_h = CG3_x.toarray()
    j = 0
    summa = 0
    while j < size:
        summa += (CG2[j]-cg_h[j][0])**2
        j = j + 1
    mer2 = summa/size
    return CG1_x, CG2,CG3_x.A, mer1,mer2, CG1_t, CG3_t, A.A, b.A,

In [15]:
CG = ans(size = 50)

Количество шагов : 25
0.00964808464050293
Количество шагов : 23
0.1418170928955078


In [16]:
A = CG[7]
B = CG[8]

In [17]:
start_time = time.time()
solve(A,B)
print(time.time()-start_time)

0.009508848190307617


In [18]:
print("Среднеквадратичная ошибка: {}".format(CG[3][0]))
print("Среднеквадратичная ошибка: {}".format(CG[4]))

Среднеквадратичная ошибка: 4.280656997972435e-10
Среднеквадратичная ошибка: 1.054741529553345e-08


In [19]:
CG = ans(size = 75)

Количество шагов : 34
0.0354008674621582
Количество шагов : 29
0.12494468688964844


In [20]:
A = CG[7]
B = CG[8]

In [21]:
start_time = time.time()
solve(A,B)
print(time.time()-start_time)

0.002691507339477539


In [22]:
print("Среднеквадратичная ошибка: {}".format(CG[3][0]))
print("Среднеквадратичная ошибка: {}".format(CG[4]))

Среднеквадратичная ошибка: 4.042087768754371e-10
Среднеквадратичная ошибка: 9.658966840006633e-08


In [23]:
CG = ans(size = 100)

Количество шагов : 61
0.04278683662414551
Количество шагов : 53
0.26924800872802734


In [24]:
A = CG[7]
B = CG[8]

In [25]:
start_time = time.time()
solve(A,B)
print(time.time()-start_time)

0.0037107467651367188


In [26]:
print("Среднеквадратичная ошибка: {}".format(CG[3][0]))
print("Среднеквадратичная ошибка: {}".format(CG[4]))

Среднеквадратичная ошибка: 5.355725141794007e-10
Среднеквадратичная ошибка: 1.0824983390299796e-07


In [27]:
print("Ответ:\n {}".format(CG[0]))
print("Точный ответ:\n {}".format(CG[1]))

Ответ:
 [[5.68615528e+01]
 [2.85993439e+01]
 [1.60278475e+01]
 [1.35275894e+01]
 [3.40400926e+01]
 [5.99333248e+00]
 [2.05370967e+01]
 [8.45763554e+01]
 [6.23992886e+01]
 [5.47267687e+00]
 [1.48761628e+01]
 [1.72161540e+01]
 [1.49196684e+00]
 [1.02738075e+01]
 [2.76368981e+01]
 [2.35954945e+00]
 [2.12268149e+01]
 [1.44608288e+01]
 [1.22366721e+01]
 [1.10207213e+01]
 [3.31257355e+01]
 [4.12453496e+01]
 [1.08137929e+01]
 [3.13439019e+01]
 [1.64837234e+01]
 [1.18662828e+00]
 [1.61945174e+01]
 [3.88264059e+01]
 [1.12462561e+00]
 [9.49512508e+00]
 [4.01008252e+01]
 [3.38434164e+01]
 [2.79592312e+01]
 [1.87349965e+01]
 [1.70644288e+00]
 [2.91208232e+01]
 [8.22823259e+00]
 [2.18241795e+01]
 [1.83831894e+01]
 [2.59058973e+00]
 [2.89475638e+00]
 [5.12055818e+00]
 [2.48060804e+01]
 [3.78051133e+00]
 [7.45216982e+00]
 [9.91623839e-01]
 [1.16925512e+01]
 [7.16313881e+00]
 [3.15163513e+00]
 [1.73119063e+01]
 [2.33229395e+01]
 [2.41957864e+00]
 [3.01147706e+01]
 [5.87330528e+01]
 [1.11253328e+01]
 [

#### Сравнение времени со встроенной функцией

In [28]:
A = CG[7]
B = CG[8]

In [29]:
start_time = time.time()
solve(A,B)
print(time.time()-start_time)

0.0007696151733398438
